# First Approach to a univariate Long-Short-Term Memory model for predicting the solar output  #

For our optimisation we need solar output predictions. In this notebook we will use a univariate Long-Short-Term Memory model to predict the solar output. 


 https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM


First install all the dependencies:

In [37]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import permutations

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf

import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense, Activation, Dropout, LSTM , Conv1D, MaxPooling1D  # create layers


from sklearn.model_selection import train_test_split
 
np.random.seed(42)
tf.random.set_seed(42)


We will start with loading the pickle file with our full dataset into this notebook. 



In [38]:
df = pd.read_pickle("../data/final_dataframe.pkl")

The column names are not very easy to work with and can be a bit hard to read. Therefore we will rename them to make them easier to read.

In [39]:
def col_names(df):
    ''' this function renames the columns to make them easier to read 
      additionally set the date as index in our dataframe'''
    column_names = {'Photovoltaics [MWh] Original resolutions': 'Solar_generation_MWh',
                'Photovoltaics [MW] Calculated resolutions': 'Solar_installed_MW',
                'Total (grid load) [MWh] Original resolutions': 'Total_consumption_MWh',
                'Germany/Luxembourg [€/MWh] Calculated resolutions': 'DE_LU_price_per_MWh',}
    df.rename(columns=column_names, inplace=True)
    df.set_index('Date', inplace=True)
    return df

col_names(df)


,Solar_generation_MWh,Solar_installed_MW,Total_consumption_MWh,DE_LU_price_per_MWh,normalisation_factor,Solar_generation_MWh_normalized
Date,,,,,,
2018-10-01 00:00:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:15:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:30:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:45:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 01:00:00,0.0,42805.0,10589.75,56.10,0.684015,0.0
...,...,...,...,...,...,...
2023-06-01 22:45:00,0.0,62579.0,12945.50,95.41,1.000000,0.0
2023-06-01 23:00:00,0.0,62579.0,12817.75,86.53,1.000000,0.0
2023-06-01 23:15:00,0.0,62579.0,12539.00,86.53,1.000000,0.0


Now we can already split the data into train and test set. Important to note here is that the shuffle has to be false otherwise the split is not appropriate for time series analysis. I will use the previously defined approach from the 20230704_train_val_test_split notebook 

In [383]:
# I have a huge problem with the 0 therfore I will add 1 to all my datapoints 
#df['Solar_generation_MWh_normalized'] = df['Solar_generation_MWh_normalized'] + 1

In [40]:
target = ['Total_consumption_MWh']

In [47]:
def test_train_timeseries(df):
    ''' In the first part we select the train and test data.
    In the second per the columns we want to use for our predictions '''
    
    test = df[df.index >= '2022-06-01']
    train = df[df.index < '2022-06-01']

    # now we select the columns we want to use for our predictions

    test = test[['Total_consumption_MWh']]
    train = train[['Total_consumption_MWh']]
    return test, train

test, train = test_train_timeseries(df)

In [385]:
#Alternatively we could also use 
#train, test = train_test_split(df[['Solar_generation_MWh_normalized']], test_size=.25, shuffle=False)

In [48]:
test

,Total_consumption_MWh
Date,
2022-06-01 00:00:00,11317.75
2022-06-01 00:15:00,11203.75
2022-06-01 00:30:00,11089.00
2022-06-01 00:45:00,10982.75
2022-06-01 01:00:00,10934.50
...,...
2023-06-01 22:45:00,12945.50
2023-06-01 23:00:00,12817.75
2023-06-01 23:15:00,12539.00


In [50]:
train

,Total_consumption_MWh
Date,
2018-10-01 00:00:00,10589.75
2018-10-01 00:15:00,10589.75
2018-10-01 00:30:00,10589.75
2018-10-01 00:45:00,10589.75
2018-10-01 01:00:00,10589.75
...,...
2022-05-31 22:45:00,12692.00
2022-05-31 23:00:00,12446.50
2022-05-31 23:15:00,12243.50


I worked nicely

In [51]:
#Let's scale the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [52]:
#Scale the data for the model #! Test MinMaxScaler 

scaler = StandardScaler()
train['Total_consumption_MWh'] = scaler.fit_transform(train[['Total_consumption_MWh']])
test['Total_consumption_MWh'] = scaler.transform(test[['Total_consumption_MWh']])

In [53]:
test

,Total_consumption_MWh
Date,
2022-06-01 00:00:00,-1.231615
2022-06-01 00:15:00,-1.277501
2022-06-01 00:30:00,-1.323690
2022-06-01 00:45:00,-1.366457
2022-06-01 01:00:00,-1.385878
...,...
2023-06-01 22:45:00,-0.576425
2023-06-01 23:00:00,-0.627846
2023-06-01 23:15:00,-0.740046


In [54]:
# split a univariate sequence into samller samples to feed into the LSTM
def split_sequence(input, n_steps, pred_size):
    x, y = list(), list()
    for i in range(len(input)):
        end_ix = i + n_steps # find the end of this pattern
        if end_ix+pred_size > len(input)-1: # check if we are beyond the sequence
            break
        seq_x, seq_y = input[i:end_ix], input[end_ix: end_ix+pred_size]# gather input and output parts of the pattern
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.squeeze(np.array(y))

In [55]:
# define input sequence
input = train['Total_consumption_MWh']
# choose a number of time steps
n_steps = 672
# prediction size 
pred_size= 96
# split into samples
X, y = split_sequence(input, n_steps, pred_size)
# summarize the data
print(len(X), len(y))


127776 127776


In [56]:
X_test

array([[[-1.23161488],
        [-1.27750133],
        [-1.32368967],
        ...,
        [-0.89913933],
        [-0.98728963],
        [-1.06467271]],

       [[-1.27750133],
        [-1.32368967],
        [-1.36645665],
        ...,
        [-0.98728963],
        [-1.06467271],
        [-1.15221923]],

       [[-1.32368967],
        [-1.36645665],
        [-1.3858779 ],
        ...,
        [-1.06467271],
        [-1.15221923],
        [-1.22155206]],

       ...,

       [[-0.98618272],
        [-1.0407232 ],
        [-1.10200577],
        ...,
        [-0.5253056 ],
        [-0.60550627],
        [-0.71690167]],

       [[-1.0407232 ],
        [-1.10200577],
        [-1.1906592 ],
        ...,
        [-0.60550627],
        [-0.71690167],
        [-0.78402068]],

       [[-1.10200577],
        [-1.1906592 ],
        [-1.25043235],
        ...,
        [-0.71690167],
        [-0.78402068],
        [-0.86542889]]])

In [57]:
X_test, y_test = split_sequence(test['Total_consumption_MWh'] , n_steps, pred_size)

In [58]:
print(y.shape, X.shape)

(127776, 96) (127776, 672)


In [59]:
#Now we have to define the validation set for our model #! I see this approach is not so useful, therefore I will use the train test split with shuffling to obtain the validation data. Here i am not loosing the lateest data for training my model 
def val_set(X,y):
    X, X_val, y, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    return X, X_val, y, y_val
    #! old approach
    #train_size = round(len(X) * 0.8)
    #X = X[:train_size, :]
    #X_val = X[train_size:, :]
    #y = y[:train_size, :]
    #y_val = y[train_size:, :]
    
X, X_val, y, y_val = val_set(X, y)

In [60]:
y_val.shape

(25556, 96)

In [61]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
 #! correction still necessary

def reshape_for_LSTM(X, y, features):
    features
    X = X.reshape((X.shape[0], X.shape[1], features))
    y = y.reshape((y.shape[0], y.shape[1]))
    return X, y

In [62]:
X, y = reshape_for_LSTM(X, y, 1)

In [63]:
X_val, y_val = reshape_for_LSTM(X_val, y_val, 1)

In [64]:
X_test, y_test = reshape_for_LSTM(X_test, y_test, 1)

In [65]:
X_val.shape

(25556, 672, 1)

In [66]:
X.shape

(102220, 672, 1)

## Lets start the modeling approach using the Long short term memory model ##



In [67]:
# Define dictionary to store results
history = {}

# Define number of epochs and learning rate decay
N_TRAIN = len(X)
EPOCHS = 50
BATCH_SIZE = 2600 # total sample size = 97593 each batch 2600 samples (49 batches ) #! has to be adjusted further to improve
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay( 
    0.001,  #! please adjust and finetune ? should be fine like this 
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)


# Define optimizer used for modelling
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule, name='Adam')  # due to a warning message I used the legacy.Adam 

In [68]:
# Define path where checkpoints should be stored
checkpoint_path = "modeling/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0) # Set verbose != 0 if you want output during training 

#create a callback to stop early once there is no improvement in the loss
cp_early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',
                                restore_best_weights=True,
                                verbose = True)

Note how many output layer are needed for predicting several timestamps? Please check one output layer is enough but some of the parameters have to be adjusted,

n_steps, n_features
X.shape[1], X.shape[2]

reason for not having activation functions https://datascience.stackexchange.com/questions/66594/activation-function-between-lstm-layers
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTMCell

output layer structure : https://stackoverflow.com/questions/46797891/output-shape-of-lstm-model#46799544

https://shiva-verma.medium.com/understanding-input-and-output-shape-in-lstm-keras-c501ee95c65e

In [406]:
#from keras.layers import Dense, Activation, Dropout, LSTM , Conv1D, MaxPooling1D, LeakyReLU

In [69]:
def get_simple_LSTM_model():
    simple_LSTM = tf.keras.Sequential([
      tf.keras.layers.Conv1D(32, kernel_size = 5, activation ='relu', input_shape =(X.shape[1], X.shape[2])),
      tf.keras.layers.MaxPooling1D(pool_size = 2),
      tf.keras.layers.LSTM(45, kernel_initializer = 'uniform', return_sequences=True), # ! units are not set in stone yet 
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.LSTM(32, return_sequences=True),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.LSTM(32, return_sequences=False),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(y.shape[1] ,kernel_initializer = 'uniform', activation='linear') #96 to predict a day 
    ])

    simple_LSTM.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanAbsolutePercentageError(), 
                  metrics=[tf.keras.losses.MeanAbsolutePercentageError()])
    return simple_LSTM

In [70]:
with tf.device('/cpu:0'):
    simple_LSTM = get_simple_LSTM_model()
    print(simple_LSTM.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 668, 32)           192       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 334, 32)           0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 334, 45)           14040     
                                                                 
 dropout_3 (Dropout)         (None, 334, 45)           0         
                                                                 
 lstm_4 (LSTM)               (None, 334, 32)           9984      
                                                                 
 dropout_4 (Dropout)         (None, 334, 32)           0         
                                                      

In [71]:
with tf.device('/cpu:0'):
    history = simple_LSTM.fit(X,
                        y,
                        batch_size= BATCH_SIZE,
                        validation_data= (X_val, y_val),   ##### probably best to make validation data D #! TO DO 
                        verbose=10,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS,
                        shuffle = False, 
                        callbacks=[cp_callback, cp_early_stop]) # try without early stopping to see if there is something wrong cp_early_stop

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Restoring model weights from the end of the best epoch: 10.
Epoch 15: early stopping


In [ ]:
#with tf.device('/cpu:0'):
#    simple_LSTM_reloaded = tf.keras.models.load_model('saved_model/simple_LSTM')
#STOP
# Check its architecture
#simple_LSTM_reloaded.summary()

NameError: name 'STOP' is not defined

In [30]:
history.history

{'loss': [100.31908416748047,
  100.20409393310547,
  100.20437622070312,
  100.14425659179688,
  100.16031646728516,
  100.14885711669922,
  100.13554382324219,
  100.1397705078125,
  100.13341522216797,
  100.13502502441406,
  100.14124298095703,
  100.1461181640625,
  100.12493133544922],
 'mean_absolute_percentage_error': [100.31906127929688,
  100.2004623413086,
  100.20076751708984,
  100.14163970947266,
  100.15731048583984,
  100.14613342285156,
  100.13301849365234,
  100.13712310791016,
  100.13079071044922,
  100.1324691772461,
  100.13842010498047,
  100.14311981201172,
  100.12239837646484],
 'val_loss': [100.19440460205078,
  100.10308074951172,
  100.12483978271484,
  100.13921356201172,
  100.10443878173828,
  100.07920837402344,
  100.05791473388672,
  100.05050659179688,
  100.06433868408203,
  100.05816650390625,
  100.07321166992188,
  100.09685516357422,
  100.08013153076172],
 'val_mean_absolute_percentage_error': [100.19107055664062,
  100.10133361816406,
  100.1

In [31]:
scores = simple_LSTM.evaluate(X, y)

3195/3195 [==============================] - 122s 38ms/step - loss: 100.0523 - mean_absolute_percentage_error: 100.0522


In [33]:
print("\n%s: %.2f%%" % (simple_LSTM.metrics_names[1], scores[1]))


mean_absolute_percentage_error: 100.05%


than we take the splited test set and let the model predict 

In [78]:
x_input = X_test
#x_input = x_input.reshape((x_input.shape[0], x_input.shape[1], 1))
y_pred = simple_LSTM.predict(x_input, verbose=0)
print(y_pred)

[[ 0.00067435 -0.00091283  0.00026331 ... -0.00023365 -0.00037922
  -0.00046914]
 [ 0.00065425 -0.00092837  0.00024467 ... -0.00024143 -0.00038393
  -0.00046925]
 [ 0.0006367  -0.00093663  0.00023033 ... -0.00024459 -0.00038029
  -0.00046591]
 ...
 [ 0.00080868 -0.00073177  0.00032274 ... -0.00010246 -0.00033343
  -0.00038295]
 [ 0.00079773 -0.00074975  0.00031009 ... -0.00011144 -0.00033706
  -0.00038844]
 [ 0.00079677 -0.00074376  0.00030729 ... -0.00010651 -0.00033229
  -0.00038228]]


In [79]:
print("Evaluate on test data")
results = simple_LSTM.evaluate(X_test, y_test, batch_size=2600)
print("test loss, test acc:", results)

Evaluate on test data
14/14 [==============================] - 21s 1s/step - loss: 100.0398 - mean_absolute_percentage_error: 100.0406
test loss, test acc: [100.039794921875, 100.0406265258789]


Lets now make a new timeseries from out predicted values so we can plot them nicely

In [76]:
# First we get all the original timestamps from our original test dataset. We splitted this dataset into features and target. 
# we used the first 672 entries to predict the next 96 so for our predicted values we will start only from index 672.
# The following timestamps will be predicted by our model 
df_trial = test.iloc[672:].copy() 

In [77]:
# Then we make a new column containing the first prediction of our
df_trial['predicted_values'] = y_pred[:, 0]

ValueError: Length of values (34368) does not match length of index (34464)

In [ ]:
df_trial

,Solar_generation_MWh_normalized
Date,
2022-06-04 00:00:00,-0.624547
2022-06-04 00:15:00,-0.624547
2022-06-04 00:30:00,-0.624547
2022-06-04 00:45:00,-0.624547
2022-06-04 01:00:00,-0.624547
...,...
2023-06-01 22:45:00,-0.624547
2023-06-01 23:00:00,-0.624547
2023-06-01 23:15:00,-0.624547


In [36]:
y_pred[:1]

NameError: name 'y_pred' is not defined

The results are still not very good. It seems that there are some problems with handeling the 0 and also the seasonality. More work is needed. 
    Option 1: I will remove the seasonality from the data before it goes into the model 
    Option 2: I will include the price and weather data to make the model more robust. 

### Let's now create a proper output table which can then be used for the Optimisation ###
Now we will transform out output back into the same units as before and add them to a dataframe. 

In [34]:
def reverse_and_frame(X, y):
    ''' The input are both our arrays X_test and the predicted y 
     1st. We will inverse transfrom the arrays to the original dimensions needed by the optimizer 
     2nd. We create a dataframe for both x_test and y_pred 
     3rd. We merge all the columns representing the timesteps into a single array in one column 
     4th. We concatenate the two dataframes into one 
    '''
    inversed_y_pred = scaler.inverse_transform(y)
    inversed_X_test = scaler.inverse_transform(X.reshape(X.shape[0], X.shape[1]))
    X_test = pd.DataFrame(inversed_X_test)
    y_pred = pd.DataFrame(inversed_y_pred)
    X_test['input_array'] = X_test.apply(lambda row: np.array(row), axis=1)
    y_pred['output_array'] = y_pred.apply(lambda row: np.array(row), axis=1)
    pred = pd.concat([X_test['input_array'],y_pred['output_array']],  axis=1)
    return pred

In [35]:
consumption_predictions= reverse_and_frame(X_test, y_pred)

NameError: name 'y_pred' is not defined

### include the date and time column back to the output dataframe ###

This is currently a dirty fix. It would eb better to use the date column from the original dataframe and concat on this.

In [ ]:
def add_time_frame(x):
    Date = ["2022-06-01 00:00:00"]
    Date[0] = pd.to_datetime(Date[0])
    Date[0] = (Date[0] + datetime.timedelta(minutes=(15 * 672)))
    for i in range(len(x)-1):
        Date.append(Date[i]+ datetime.timedelta(minutes=15))
    x['Date'] = Date
    return x

In [ ]:
df_w_date = add_time_frame(solar_predictions)

In [ ]:
df_w_date.describe()

,Date
count,34752
mean,2022-12-05 23:52:30
min,2022-06-08 00:00:00
25%,2022-09-06 11:56:15
50%,2022-12-05 23:52:30
75%,2023-03-06 11:48:45
max,2023-06-04 23:45:00


In [ ]:
df_solar_predictions = df_w_date[['Date', 'output_array']].copy()

In [1]:
df_solar_predictions.to_pickle("../predictions/solar_predictions.pkl")

NameError: name 'df_solar_predictions' is not defined

In [ ]:
# Save the entire small model as a SavedModel.
#!mkdir -p ../models/saved_model
simple_LSTM.save('../models/saved_model/simple_LSTM_2')

INFO:tensorflow:Assets written to: ../models/saved_model/simple_LSTM_2/assets


INFO:tensorflow:Assets written to: ../models/saved_model/simple_LSTM_2/assets


### Short summary ###
model is performing poorly, the overall MAPE is still around 65 % error which is helpful for our approach. Based on the output I is not able to handle the 0 very well. I will now look a bit deeper into the 0 problem. 

### Alternative approach get rid of the seasonality beforehand 